In [3]:
from cadcad.spaces import space
from cadcad.spaces import Bit, Real, Integer
from cadcad.trajectories import Point

The Purpose of this notebook is to experiment with spaces and points

In [4]:
R2 = Real**2

R2dims = list(R2.dimensions().keys())
R2dims

['real_0', 'real_1']

In [5]:
R2 = R2.rename_dims({R2dims[0]:'x', R2dims[1]:'y'})

R2.dimensions()

{'x': 'Real', 'y': 'Real'}

In [6]:
@space
class Bot:
    position: R2 #location of the bot on a 2D grid
    velocity: R2 #rate of change of position on a 2D grid
    radius: float #how big around is the robot
    is_working: bool #if the robot crashes (into another robot) they cease to function

@space
class BotArray:
    bots: list[Bot]

In [7]:
BotArray.unroll_schema()

{'bots': 'list'}

It seems to me that we shoukd change the way `unroll_schema()` handles lists to incude details about the class it is a list of. 

In [8]:
Bot.unroll_schema()

{'position': {'x': {'real': 'float'}, 'y': {'real': 'float'}},
 'velocity': {'x': {'real': 'float'}, 'y': {'real': 'float'}},
 'radius': 'float',
 'is_working': 'bool'}

Now let's make some points

In [9]:
Real.unroll_schema()

{'real': 'float'}

In [13]:
Real.schema()

AttributeError: type object 'Real' has no attribute 'schema'

In [11]:
zero = Point(Real, {'real':0.0})

Print(zero)

AttributeError: type object 'Real' has no attribute 'schema'

In [12]:
zero.data

NameError: name 'zero' is not defined

In [10]:
def R2point(x,y):
    xpt_data = {'real':x}
    ypt_data = {'real':y}
    return Point(R2,{'x': xpt_data, 'y': ypt_data},check_types=False)

OO = R2point(0.0,0.0) 

print(OO)

Point in space 2-Real has data
{
    "x": {
        "real": 0.0
    },
    "y": {
        "real": 0.0
    }
}



In [11]:
OO.data

{'x': {'real': 0.0}, 'y': {'real': 0.0}}

In [12]:
def init_Bot(pos=Point[R2], vel=Point[R2]):
    return Point(Bot, {'position':pos.data, 'velocity':vel.data, 'radius':.01, 'is_working':True }, check_types=False)

bot_naught = init_Bot(OO, OO)

In [13]:
print(bot_naught)

Point in space Bot has data
{
    "position": {
        "x": {
            "real": 0.0
        },
        "y": {
            "real": 0.0
        }
    },
    "velocity": {
        "x": {
            "real": 0.0
        },
        "y": {
            "real": 0.0
        }
    },
    "radius": 0.01,
    "is_working": true
}



In [14]:
import numpy as np
n = 3
starting_positions = [R2point(float(np.random.randn()),float(np.random.randn())) for i in range(n)]

print(starting_positions)
print("")
for ipos in starting_positions:
    print(ipos)


[<cadcad.trajectories.Point object at 0x7fa910e9e8c0>, <cadcad.trajectories.Point object at 0x7fa9103a7d90>, <cadcad.trajectories.Point object at 0x7fa9103b8370>]

Point in space 2-Real has data
{
    "x": {
        "real": -1.2481267811480887
    },
    "y": {
        "real": -0.212265770270742
    }
}

Point in space 2-Real has data
{
    "x": {
        "real": 0.7284392879153955
    },
    "y": {
        "real": 1.4198095459635094
    }
}

Point in space 2-Real has data
{
    "x": {
        "real": -2.321318575526753
    },
    "y": {
        "real": -1.7874270196950064
    }
}



In [15]:

def init_Bots(starting_positions=list[Point[R2]]):
    data = [init_Bot(ipos, OO) for ipos in starting_positions]
    return Point(BotArray, data)

myBotArray = init_Bots(starting_positions)
print(myBotArray)

TypeError: Point's data must be a dictionary

In [17]:
def init_Bots(starting_positions=list[Point[R2]]):
    bots = [init_Bot(ipos, OO) for ipos in starting_positions]
    return Point(BotArray, [b.data for b in bots])

myBotArray = init_Bot(starting_positions)
print(myBotArray)

AttributeError: 'list' object has no attribute 'data'

It seems like the behavior of lists as part of spaces is a little tricky -- for example printing schema's doesn't handle lists clearly and when i use `myPoint.data` on dimensions with Type `list[MySpace]` it fails to get the data out. I think we might need some extra machinery for handling dimesions like `list[MySpace]`.